<a href="https://colab.research.google.com/github/wonyoung-ji/NLP/blob/master/6%EC%9D%BC%EC%B0%A8/%EB%82%98%EC%9D%B4%EB%B8%8C%EB%B2%A0%EC%9D%B4%EC%A7%80%EC%95%88_%EB%84%A4%EC%9D%B4%EB%B2%84%EC%98%81%ED%99%94%EB%A6%AC%EB%B7%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! sudo apt-get install g++ openjdk-7-jdk #Install Java 1.7+ 
#!sudo apt-get install python-dev; pip install konlpy # Python 2.x 
!sudo apt-get install python3-dev; pip3 install konlpy # Python 3.x 
!sudo apt-get install curl 
! bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

데이터 DL > https://github.com/e9t/nsmc

## 01. 데이터 불러오기 및 결측치처리

In [91]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [276]:
import pandas as pd
train = pd.DataFrame(pd.read_csv('/content/gdrive/My Drive/NLP/네이버영화리뷰실습/ratings_train.txt', sep='\t', quoting=3,encoding='utf-8')) 
#label: 0 = 부정, 1 = 긍정

In [277]:
train.dropna(inplace=True)
train.reset_index(inplace=True)

In [278]:
# del train['index']
train

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149990,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149991,8549745,평점이 너무 낮아서...,1
149992,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149993,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


## 02. 토큰화

In [345]:
#토큰화
from konlpy.tag import Mecab
mecab = Mecab()
mecab_token=[]
for i, token in enumerate(train['document']):
  mecab_token.append(mecab.morphs(token))
mecab_token[:3]

[['아', '더', '빙', '.', '.', '진짜', '짜증', '나', '네요', '목소리'],
 ['흠',
  '.',
  '..',
  '포스터',
  '보고',
  '초딩',
  '영화',
  '줄',
  '.',
  '...',
  '오버',
  '연기',
  '조차',
  '가볍',
  '지',
  '않',
  '구나'],
 ['너무', '재', '밓었다그래서보는것을추천한다']]

## 품사부착

In [346]:
#품사부착
mecab_pos=[]
for token in train['document']:
  mecab_pos.append(mecab.pos(token))
mecab_pos[:5]

[[('아', 'IC'),
  ('더', 'MAG'),
  ('빙', 'MAG'),
  ('.', 'SF'),
  ('.', 'SY'),
  ('진짜', 'MAG'),
  ('짜증', 'NNG'),
  ('나', 'NP'),
  ('네요', 'VCP+EC'),
  ('목소리', 'NNG')],
 [('흠', 'IC'),
  ('.', 'SF'),
  ('..', 'SY'),
  ('포스터', 'NNP'),
  ('보고', 'JKB'),
  ('초딩', 'NNG'),
  ('영화', 'NNG'),
  ('줄', 'NNG'),
  ('.', 'SF'),
  ('...', 'SY'),
  ('오버', 'NNG'),
  ('연기', 'NNG'),
  ('조차', 'JX'),
  ('가볍', 'VA'),
  ('지', 'EC'),
  ('않', 'VX'),
  ('구나', 'EC')],
 [('너무', 'MAG'), ('재', 'XPN'), ('밓었다그래서보는것을추천한다', 'UNKNOWN')],
 [('교도소', 'NNG'),
  ('이야기', 'NNG'),
  ('구먼', 'VCP+EF'),
  ('.', 'SF'),
  ('.', 'SY'),
  ('솔직히', 'MAG'),
  ('재미', 'NNG'),
  ('는', 'JX'),
  ('없', 'VA'),
  ('다', 'EF'),
  ('.', 'SF'),
  ('.', 'SY'),
  ('평점', 'NNG'),
  ('조정', 'NNG')],
 [('사이몬페그', 'NNP'),
  ('의', 'JKG'),
  ('익살', 'NNG'),
  ('스런', 'XSA+ETM'),
  ('연기', 'NNG'),
  ('가', 'JKS'),
  ('돋보였', 'VV+EP'),
  ('던', 'ETM'),
  ('영화', 'NNG'),
  ('!', 'SY'),
  ('스파이더맨', 'NNP'),
  ('에서', 'JKB'),
  ('늙', 'VV'),
  ('어', 'EC'),
  ('보이', 'VV'),
  ('기',

In [7]:
# from collections import Counter
# Counter(train['document']).most_common()

## 불용어처리

In [259]:
#불용어처리
stopPos = ['JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JC','JX',   #조사
          #  'SF','SE','SSO','SSC','SC','SH','SL','SN','SY',     #기타부호
           'VCP','VCN',
           'EP','EF','ETN','ETM',    #어미
           'XSN','XSV','XSA','XPN',  # 접미사 및 접두사
           ]
stopWord = ['네요','그랬','한','는','이','다','줄','지','하','서','년','거','야','어','할',]


X_train=[]
for tags in mecab_pos:
  word = []
  for tag in tags:
    if tag[1] not in stopPos:
      if tag[0] not in stopWord:
        word.append(tag[0])
  X_train.append(' '.join(word))

## 학습


In [314]:
Y_train = list(train['label'])
print(X_train[:30])
print(Y_train[:30])

['아 더 빙 . . 진짜 짜증 나 목소리', '흠 . .. 포스터 초딩 영화 . ... 오버 연기 가볍 않 구나', '너무 밓었다그래서보는것을추천한다', '교도소 이야기 구먼 . . 솔직히 재미 없 . . 평점 조정', '사이몬페그 익살 스런 연기 돋보였 영화 ! 스파이더맨 늙 보이 했 커스틴 던스트 너무나 이뻐 보였', '막 걸음마 뗀 3 세 초등 학교 1 학년 인 8 살 영화 . ㅋㅋㅋ . .. 별반 개 아까움 .', '원작 긴장감 제대로 살려 내 못했 .', '별 반개 아깝 욕 나온다 이응경 길용우 연기 생활 몇 인지 . . 정말 발 해도 그것 보단 낫 납치 . 감금 반복 반복 . . 드라마 가족 없 연기 못 사람 모엿 네', '액션 없 는데 재미 있 몇 안 되 영화', '왜 케 평점 낮 건데 ? 꽤 볼 만한데 . . 헐리우드 화려 함 너무 길들여져 있 ?', '걍인피니트가짱이다 . 진짜 짱 ♥', '볼 때 눈물나 죽 90 대 향수 자극 ! ! 허진호 감성 절제 멜로 달인 ~', '울 면서 손들 고 횡단 보도 건널 때 뛰쳐나올 뻔 이범수 연기 드럽 게 못해', '담백 고 깔끔 해서 좋 . 신문 기사 보다 보 면 자꾸 잊어버린다 . 그 사람 것 .', '취향 존중 한다지만 진짜 내생 극장 본 영화 중 가장 잼 감동 임 스토리 거지 고 감동 어거지', 'ㄱ 냥 매번 긴장 고 밋 음 ㅠㅠ', '참 사람 웃긴 게 바스코 이기면 락스 코 라고 까 고 바비 이기 면 아이돌 라고 깐다 . 그냥 까 고 싶 어서 안달 난 것 보인다', '굿바이 레닌 표절 인 것 이해 는데 왜 뒤 갈수록 재미없 냐', '이건 정말 깨알 캐스팅 질퍽 않 산뜻 내용 구성 잘 버 무 러진 깨알 일 드 ! !♥', '약탈자 위한 변명 , 이라 . 저놈 착한 놈 절대 아닌걸요 .', '나름 심오 뜻 있 듯 . 그냥 학생 선생 놀아나 영화 절대 아님', '보 면서 웃 않 건 가능', '재미없 지루 고 . 같 음식 영화 인데 바베트 만찬 고 넘 차이 . ... 바베트 만찬 이야기 있 고 음식 보 재

In [315]:
from sklearn.feature_extraction.text import CountVectorizer     #빈도수세기
from sklearn.naive_bayes import MultinomialNB                   #나이브베이지안 구하기

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
clf = MultinomialNB().fit(X_train_counts, Y_train)      #fit(a,b):a를 b에 매칭

## 검정

In [334]:
test = pd.DataFrame(pd.read_csv('/content/gdrive/My Drive/NLP/네이버영화리뷰실습/ratings_test.txt', sep='\t', quoting=3,encoding='utf-8')) 

In [335]:
# test.dropna(inplace=True)
# test.reset_index(inplace=True)

In [330]:
#품사 태깅
test_pos = []
for document in test['document']:
  test_pos.append(mecab.pos(document))

In [336]:
#불용어처리
stopPos = ['JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JC','JX',   #조사
           'SF','SE','SSO','SSC','SC','SH','SL','SN','SY',     #기타부호
           'VCP','VCN',
           'EP','EF','ETN','ETM',    #어미
           'XSN','XSV','XSA','XPN',  # 접미사 및 접두사
           ]
stopWord = ['네요','그랬','한','는','이','다','줄','지','하','서','년','거','야','어','할',]


X_test=[]
for tags in test_pos:
  word = []
  for tag in tags:
    if tag[1] not in stopPos:
      if tag[0] not in stopWord:
        word.append(tag[0])
  X_test.append(' '.join(word))

In [347]:
X_test[:5]

['굳 ㅋ',
 '',
 '뭐 평점 나쁘 진 않 지만 점 더더욱 잖아',
 '지루 않 은데 완전 막장 임 돈 주 고 보',
 '어도 별 다섯 개 줬 텐데 왜 나와서 제 심기 불편 게']

In [343]:
test['acc'] = 0
for i, document in enumerate(X_test):
  prob=clf.predict_proba(count_vect.transform([document]))
  if prob[0][0] <= prob[0][1]:
    test['acc'][i] += 1
test

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,index,id,document,label,acc
0,0,6270596,굳 ㅋ,1,0
1,1,9274899,GDNTOPCLASSINTHECLUB,0,0
2,2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0,0
3,3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0,0
4,4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0,0
...,...,...,...,...,...
49992,49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1,1
49993,49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0,0
49994,49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0,1
49995,49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0,0


In [340]:
accuracy = 0
for i in range(len(test)):
  if test['label'][i] == test['acc'][i]:
    accuracy += 1
accuracy/len(test)

0.8083885033101986